# Ejercicio Industria 4.0

En este ejercicio se intentará obtener unos resultados a partir de un dataset muy famoso de clasificación de música de spotify. Todos los imports necesarios ya estan añadidos en el siguiente código.

### Todos los campos a modificar se identificaran por el simbolo @.


In [2]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn import preprocessing
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

## Paso 1: Leer el dataset

In [2]:
dataset = pd.read_csv(@)

## Paso 2: Revisar los campos

In [ ]:
@

## Paso 3: Eliminar filas no utiles + filas con nulos

Para ello, hay que revisar que puede significar cada fila y identificar aquellas que no nos sirvan.

In [4]:
columnasNoUtiles = [@, ...]
dataset = dataset.drop(columns=columnasNoUtiles, inplace=False)

Investigando el dataset nos hemos dado cuenta que aparecen simbolos "?" para codificar algunos missings.

Elimina todas las filas que tengan este valor.

In [ ]:
dataset = dataset.dropna()
dataset = dataset[~dataset.apply(lambda row: @ in row.values, axis=1)]
# Coger el numero de clases
nClases = len(dataset.music_genre.unique())

## Paso 4: Transformar columnas categoricas en numericas

In [5]:
columnasCategoricas = [@, ...]
dataset = pd.get_dummies(dataset, columns=columnasCategoricas)

Revisar que efectivamente la conversión ha funcionado

In [ ]:
@

## Paso 5: Extraer la columna objetivo

In [6]:
targetCol = @

Y = dataset[targetCol]
X = dataset.drop([targetCol],axis=1)

## Paso 6: Particionar entre train i test

Haz una partición entre train i test manteniendo solamente el 25% de los datos en test.

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(@, @, test_size=@, shuffle=True, stratify=Y)

Comprueba los tamaños de las particiones.

In [ ]:
len(X_train), len(X_test), len(Y_train), len(Y_test)

## Paso 7: Normalizar los datos

In [8]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Paso 8: Seleccionar y probar modelos

### SVM's

Probar los kernels 'linear' y 'rbf'. Este último es más lento. Entrena el modelo.

In [ ]:
knc = SVC(kernel=@)
knc.fit(@, @)

Predice con los datos de test

In [ ]:
pred = knc.predict(@)
print(sklearn.metrics.confusion_matrix(Y_test,pred))
print(sklearn.metrics.accuracy_score(Y_test,pred))
print(sklearn.metrics.classification_report(Y_test,pred))

### Random forest

Prueba con 100 y 500 estimadores. Entrena el modelo.

In [ ]:
rf = RandomForestClassifier(n_estimators = @)
rf.fit(@, @)

Predice con los datos de test

In [ ]:
pred = rf.predict(@)
print(sklearn.metrics.confusion_matrix(Y_test,pred))
print(sklearn.metrics.accuracy_score(Y_test,pred))
print(sklearn.metrics.classification_report(Y_test,pred))

### Redes neuronales

Primero de todo hay que convertir la salida con One Hot Encoding.

In [11]:
Y_train = pd.get_dummies(pd.DataFrame(Y_train))
Y_test = pd.get_dummies(pd.DataFrame(Y_test))

Crea una red neuronal con 2 capas internas. La primera de ellas con 128 neuronas y la segunda con 64. Recuerda terminar con una capa con el numero de clases a clasificar.

In [12]:
model = Sequential()
model.add(Dense(@, input_shape=(len(X.columns), ), activation='linear'))
model.add(Dense(@, activation='linear'))
model.add(Dense(@, activation='linear'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               3328      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
Total params: 12,234
Trainable params: 12,234
Non-trainable params: 0
_________________________________________________________________


Entrena durante 15 epocas con una partición de validación de 0.1.

In [ ]:
epochs = @
validation_split = @
history = model.fit(@, @, epochs=epochs, validation_split=validation_split)

### Parece que functiona muy mal. A que se debe esto?
Las capas no tienen activación == activación linear.

Repite la red pero prueba a cambiar las functiones de activation de la red para que sean dos relu al principio y una softmax al final.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(@, input_shape=(len(X.columns), ), activation=@))
model.add(Dense(@, activation=@))
model.add(Dense(@, activation=@))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Entrena durante 50 epocas con una partición de validación de 0.1.

In [ ]:
epochs = @
validation_split = @
history = model.fit(@, @, epochs=epochs, validation_split=validation_split)

## Enseña las curvas de aprendizaje

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(history.history['loss'], label="Training loss")
plt.plot(history.history['val_loss'], label="Validation loss")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(history.history['accuracy'], label="Training accuracy")
plt.plot(history.history['val_accuracy'], label="Validation accuracy")
plt.legend()
plt.show()

## Evaluar el modelo

### Porque se realizan los argmax en la predicción y en la target de partición de test? @

In [ ]:
pred = model.predict(X_test)

### Argmax's ###
pred = np.argmax(pred, axis=1)
y_test = np.argmax(Y_test.values, axis=1)

print(sklearn.metrics.confusion_matrix(y_test,pred))
print(sklearn.metrics.accuracy_score(y_test,pred))
print(sklearn.metrics.classification_report(y_test,pred))